In [1]:
from collections import defaultdict
import numpy as np

import torch
import torch.nn as nn

In [29]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [7]:
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [15]:
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
PAD = w2i["<pad>"]

train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/dev.txt"))

n_words = len(w2i)
n_classes = len(t2i)

In [16]:
print(f"Number of words in the vocab: {n_words}")
print(f"Number of classes: {n_classes}")

Number of words in the vocab: 17615
Number of classes: 5


In [80]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_classes):
        super(RNNModel, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=vocab_sizes, embedding_dim=embedding_dim)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, nonlinearity="relu", batch_first=True)
        self.linear = nn.Linear(hidden_dim, n_classes)
        
    def forward(self, words):
        embedded_inputs = self.embedding(words) # Size: batch_size * sentence_length * embedding_dim
        outputs_rnn, hidden_rnn = self.rnn(embedded_inputs) # Size: Batch_size * sentence_length * hidden_dim - Size: Batch_size * 1 * hidden_dim
        outputs_linear = self.linear(hidden_rnn) # Size: batch_size, hidden_dim, n_classes
        return outputs_linear.view(-1)


In [81]:
rnn_model = RNNModel(n_words, 100, 10, 5)

In [82]:
rnn_model

RNNModel(
  (embedding): Embedding(17615, 100)
  (rnn): RNN(100, 10, batch_first=True)
  (linear): Linear(in_features=10, out_features=5, bias=True)
)

In [83]:
for words, sent_class in train:
    words_tensor = torch.tensor(words).reshape(1, -1)
    # train_tensor = torch.tensor([train])
    outout = rnn_model(words_tensor)
    break

In [84]:
outout

tensor([-0.5249,  0.1665, -1.0410, -0.1782, -0.2054], grad_fn=<ViewBackward0>)

# Note: 
For the brave soul (and the one that has enough time to play with this), as I don't have enough time to preprocess the data, I'm going to stop here but keep these points in mind:

1- Sentences in the same batch need to have the same length, so pad them with <PAD>
2- Use cross-entropy as the loss function as the outputs correspond to logits
3- The goal of this notebook was just to implement a vanilla RNN and not to get the dimensions mixed up